In [3]:
import numpy as np
import pycolmap
import torch
import torch.nn as nn

from splat.gaussians import Gaussians
from splat.read_utils.read_gs_ply_files import convert_to_gaussian_schema, read_ply_file
from splat.utils import read_images_binary, read_images_text

# stem = "/Users/derek/Desktop/intro_to_gaussian_splatting"
stem = "/home/da2986/intro_to_gaussian_splatting"

# models_path = "/Users/derek/Downloads/models/"
models_path = "/home/da2986/intro_to_gaussian_splatting/models"
ply_path = f"/home/da2986/gaussian-splatting/truck/point_cloud/iteration_30000/point_cloud.ply"
vertices = read_ply_file(ply_path)
gaussians = convert_to_gaussian_schema(vertices)

colmap_path = f"{stem}/data/treehill/sparse/0"
colmap_path = f"/home/da2986/gaussian-splatting/tandt/truck/sparse/0"
reconstruction = pycolmap.Reconstruction(colmap_path)

points3d = reconstruction.points3D
images = read_images_binary(f"{colmap_path}/images.bin")
cameras = reconstruction.cameras

[-0.37234583497047424, -0.41850927472114563, -0.25019344687461853]
tensor([0.3950, 0.3819, 0.4294])


In [4]:
from splat.image import GaussianImage
from splat.utils import (
    build_rotation,
    get_extrinsic_matrix,
    getIntinsicMatrix,
    read_camera_file,
    read_image_file,
)

camera_dict = read_camera_file(colmap_path)
image_dict = read_image_file(colmap_path)
images = {}
for idx in image_dict.keys():
    image = image_dict[idx]
    camera = camera_dict[image.camera_id]
    image = GaussianImage(camera=camera, image=image)
    images[idx] = image


image_num = 2
rotation_matrix = build_rotation(torch.Tensor(image_dict[image_num].qvec).unsqueeze(0))
translation = torch.Tensor(image_dict[image_num].tvec).unsqueeze(0)
extrinsic_matrix = get_extrinsic_matrix(rotation_matrix, translation).T
focal_x, focal_y = camera_dict[image_dict[image_num].camera_id].params[:2] / 2
focal_x = int(focal_x)
focal_y = int(focal_y)
c_x, c_y = camera_dict[image_dict[image_num].camera_id].params[2:4]
width = camera_dict[image_dict[image_num].camera_id].width / 2
width = int(width)
height = camera_dict[image_dict[image_num].camera_id].height / 2
height = int(height)
intrinsic_matrix = getIntinsicMatrix(focal_x, focal_y, height, width).T

In [5]:
from splat.render_engine.gaussianScene2 import GaussianScene2

scene = GaussianScene2(gaussians=gaussians)
scene.device = "cuda"
TILE_SIZE = 16

processed_gaussians = scene.preprocess(
    extrinsic_matrix=extrinsic_matrix,
    intrinsic_matrix=intrinsic_matrix,
    focal_x=focal_x,
    focal_y=focal_y,
    width=width,
    height=height,
    tile_size=TILE_SIZE
)

covariance_2d tensor([[[ 5.2790, -1.3900],
         [-1.3900,  0.7212]]], device='cuda:0')
mid tensor([3.0001], device='cuda:0')
det tensor([1.8751], device='cuda:0')
lambda1, lambda2 tensor([5.6695], device='cuda:0') tensor([0.3307], device='cuda:0')
torch.Size([2, 2541226]) torch.Size([2, 2541226])


In [6]:
processed_gaussians.means_3d

tensor([[208.2664, 416.2477,   3.6025],
        [430.7468, 198.2466,   5.2904],
        [256.0930, 152.0372,   4.3163],
        ...,
        [621.2864, 340.0043, -25.8092],
        [839.2755,  82.5251,  23.7538],
        [-31.8114, 106.8326,  28.8077]], device='cuda:0')

In [7]:
with torch.no_grad():
    output_image = scene.render_cuda(
        preprocessed_gaussians=processed_gaussians, height=height, width=width, tile_size=TILE_SIZE
    )

62 35
Number of elements in image tensor: 1599030


In [8]:
output_image[417, 208]


tensor([0., 0., 0.], device='cuda:0')

In [6]:
from PIL import Image

image = output_image.cpu().numpy() * 255
image = Image.fromarray(image.astype(np.uint8))
image.save("test_output.jpg")

In [9]:
from PIL import Image

image_path = "/Users/derek/Desktop/intro_to_gaussian_splatting/data/treehill/images/_DSC8973.JPG"
image = Image.open(image_path)

plt.imshow(image)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/derek/Desktop/intro_to_gaussian_splatting/data/treehill/images/_DSC8973.JPG'